# Batch metadata creation for hazard rasters

This notebook will create a metadata xml file for each geotiff file in a given folder. It takes an existing XML metadata file and updates the creation date and bounding extents based on the geotiff files. 

You need to create an XML metadata file in ArcGIS Pro, with at the very minimum a bounding box and a title element. 

In [9]:
import os
import xml.etree.ElementTree as ET
from osgeo import gdal
from files import flModDate
gdal.UseExceptions()


In [10]:
XMLTEMPLATE = r"X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP1_smooth\e113.3512s21.0393.tif.xml"
DATEFMT = "%Y-%m-%dT%H:%M:%S"
filename = r"X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP1_smooth\e113.3512s21.0393.tif"

In [11]:
def get_geotiff_extent(geotiff_file: str):
    """
    Get extent of a GeoTIFF file

    :param str geotiff_file: Path to the geoTIFF file
    :return: Extents of the file in the native coordinate system
    :rtype: tuple
    """
    # Open the GeoTIFF file
    dataset = gdal.Open(geotiff_file)

    if not dataset:
        print("Error: Unable to open GeoTIFF file.")
        return None

    # Get the geotransform information
    geotransform = dataset.GetGeoTransform()

    # Get raster dimensions
    width = dataset.RasterXSize
    height = dataset.RasterYSize

    # Calculate the bounding box coordinates
    west_bl = geotransform[0]
    north_bl = geotransform[3]
    east_bl = geotransform[0] + width * geotransform[1]
    south_bl = geotransform[3] + height * geotransform[5]

    # Close the dataset
    dataset = None

    return west_bl, east_bl, south_bl, north_bl


In [12]:
def updateMetadata(geotiff, title):
    """
    Update the metadata for a GeoTIFF file, by updating the template and writing out to a new xml file

    :param str geotiff: Path to a GeoTIFF file
    :param str title: Title of the dataset
    """
    
    geotiff_extent = get_geotiff_extent(geotiff)
    # Parse the XML file
    tree = ET.parse(XMLTEMPLATE)
    root = tree.getroot()

    # Update dataIdInfo/idCitation/resTitle
    res_title_element = root.find(".//dataIdInfo/idCitation/resTitle")
    if res_title_element is not None:
        res_title_element.text = title

    createDate_element = root.find(".//dataIdInfo/idCitation/date/createDate")
    if createDate_element is not None:
        createDate_element.text = flModDate(geotiff, DATEFMT)

    # Update dataExt/geoEle/GeoBndBox/westBL
    west_bl_element = root.find(".//dataExt/geoEle/GeoBndBox/westBL")
    if west_bl_element is not None:
        new_west_bl_value = geotiff_extent[0]
        west_bl_element.text = str(new_west_bl_value)

    east_bl_element = root.find(".//dataExt/geoEle/GeoBndBox/eastBL")
    if east_bl_element is not None:
        new_east_bl_value = geotiff_extent[1]
        east_bl_element.text = str(new_east_bl_value)

    north_bl_element = root.find(".//dataExt/geoEle/GeoBndBox/northBL")
    if north_bl_element is not None:
        new_north_bl_value = geotiff_extent[2]
        north_bl_element.text = str(new_north_bl_value)

    south_bl_element = root.find(".//dataExt/geoEle/GeoBndBox/southBL")
    if south_bl_element is not None:
        new_south_bl_value = geotiff_extent[3] 
        south_bl_element.text = str(new_south_bl_value)

    # Save the updated XML to a new file
    new_xml_file = f"{geotiff}.xml"
    tree.write(new_xml_file, encoding="utf-8", xml_declaration=True)

In [14]:
INPUTPATH=r"X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP1_smooth"
title = "1-year return period local wind hazard data"
for file in os.listdir(INPUTPATH):
    basename, ext = os.path.splitext(file)
    if ext == '.tif':
        updateMetadata(os.path.join(INPUTPATH, file), title)

In [26]:
BASEPATH = r"X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local"
for root, dir, files in os.walk(BASEPATH):
    if len(files) != 0:
        print(f"Root: {root}")
        retper = os.path.split(root)[1].lstrip("RP").rstrip("_smooth")
        title = f"{retper}-year return period local wind hazard data"
        for file in files:
            basename, ext = os.path.splitext(file)
            if ext == '.tif':
                updateMetadata(os.path.join(root, file), title)


Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP10000_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP1000_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP100_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP10_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP1_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP2000_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP200_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP20_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2. DATA\1. Work Unit Assessment\NWRA\hazard\local\RP2500_smooth
Root: X:\georisk\HaRIA_B_Wind\projects\acs\2.